In [1]:
## Package imports ##
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import pandas as pd
import geopandas as gpd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


#Check shapely speedups are enabled
from shapely import speedups
speedups.enabled

#Set geopandas settings
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gpd.io.file.fiona.drvsupport.supported_drivers

{'AeronavFAA': 'r',
 'ARCGEN': 'r',
 'BNA': 'raw',
 'DXF': 'raw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'GeoJSON': 'rw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'rw',
 'GML': 'raw',
 'GPX': 'raw',
 'GPSTrackMaker': 'raw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'r',
 'S57': 'r',
 'SEGY': 'r',
 'SUA': 'r',
 'TopoJSON': 'r',
 'KML': 'rw'}

## Data ingestion

Ingest data from the International Seismological Centre (ISC)

In [ ]:
# ISC web search params
start_time = datetime(1970,1,1)
end_time = datetime(2021, 6, 21)
min_latitude = -25
max_latitude = -11
min_longitude = -80
max_longitude = -66
min_mag = 4.5
max_mag = None
min_depth = None
max_depth = None
print(start_time, end_time)

In [ ]:
### ISC Catalog stepwise search ###
# ObsPy plugin breaks when searching for too many events, so we perform search in steps of 1 year
# Start search
t1 = start_time
t2 = t1 + relativedelta(years=1)
#print('Processing:', t1, t2)
client = Client("IRIS")
# Initialise catalog
cat_init = client.get_events(starttime=t1,endtime=t2,
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")
# Set up loop for stepwise search
t1=t2
t2+=relativedelta(years=1)
#print('Beginning loop', t1, t2)
cat = cat_init
while t2 < end_time:
    try:
        #print('Loop Processing', t1, t2)
        catalogue = client.get_events(starttime=t1,endtime=t2,
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")
        cat=cat.__add__(catalogue)
        t1=t2
        t2+=relativedelta(years=1)
    except:
        import sys
        print("Oops!", sys.exc_info()[0], "occurred at ", t1, " - ", t2)
        print('FDSN Web Search failure - finalising catalog...')
        final_cat = cat
        break
    
# Add final time step and add to main catalog    
assert t1 < end_time    
try:
    cat1 = client.get_events(starttime=t1,endtime=end_time,
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")
    final_cat = cat.__add__(cat1)
    print('Final cat', final_cat)
except:
    import sys
    print("Reminder:", sys.exc_info()[0], "occurred.")
    print('FDSN Web Search failure - catalog now finalised.')

In [ ]:
#print(final_cat.__str__(print_all=True))
print(final_cat)

In [ ]:
### Small test catalog ###
client = Client("IRIS")
cat = client.get_events(starttime=UTCDateTime("2008-01-01"),endtime=UTCDateTime("2013-01-01"),
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")
print(cat)

In [ ]:
final_cat.plot(projection="local", label=None, method="cartopy", title="")
plt.show()

In [ ]:
## CREATE CATALOG DATAFRAME ##
# Create empty lists
year = []
month = []
day = []
hour = []
minute = []
second = []
lat = []
lon = []
dep = []
mag = []
time = []

# Loop over each event in the catalogue
for event in final_cat: 
    year.append(event.origins[0].time.year)
    month.append(event.origins[0].time.month)
    day.append(event.origins[0].time.day)
    hour.append(event.origins[0].time.hour)
    minute.append(event.origins[0].time.minute)
    second.append(event.origins[0].time.second)
    lat.append(event.origins[0].latitude)
    lon.append(event.origins[0].longitude)
    dep.append(event.origins[0].depth)
    mag.append(event.magnitudes[0].mag)

data = pd.DataFrame(np.array([year, month, day, hour, minute, second, lat, lon, dep, mag]).T, 
             columns=["year", "month", "day", "hour", "minute", "second",
                      "lat", "lon", "depth_km", "mag"])
data["datetime"] = pd.to_datetime(data[['year', 'month', 'day', 'hour', 'minute', 'second']])
data.head()

#Fix dtypes
catalog = data.infer_objects()
catalog.dtypes
catalog.loc[:, 'depth_km'] *=0.001
catalog.head()

In [ ]:
## Prepare geodataframe ##
# Define a geodataframe using the EQ catalog from above
catalog_gdf = gpd.GeoDataFrame(catalog, geometry=gpd.points_from_xy(catalog.lon, catalog.lat))
catalog_gdf.head()
catalog_gdf = targets.set_crs("EPSG:4326")
#catalog_gdf.head(-1)
catalog_gdf.count()
#print(targets)

In [ ]:
#plt.hist(catalog["mag"],log=True)

## Epidemic Type Aftershock Sequence (ETAS) Declustering

In [ ]:
# Haversine formula for computing spherical distances
def hav(theta):
    return np.square(np.sin(theta / 2))
def haversine(lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2, earth_radius=6.3781e3):
    # to calculate distance on a sphere
    d = 2 * earth_radius * np.arcsin(
        np.sqrt(
            hav(lat_rad_1 - lat_rad_2)
            + np.cos(lat_rad_1)
            * np.cos(lat_rad_2)
            * hav(lon_rad_1 - lon_rad_2)))
    return d

### ETAS Implementation by Marsan et al. (2017)

Omori Law parameters are fixed as follows:
$$ \alpha = 2, p = 1, c = 10^{-3} days, \gamma = 2 $$ <br>

Algorithm can be described as follows: <br>
1. Compute triggering rate from catalog as $\nu(x_i,y_i,t_i)/K $ <br>
2. Compute background rate ($ \mu(x_i, y_i, t_i) $), under the assumption of $\omega_i = 0.5$
3. Compute initial total rate as $ \lambda(x_i,y_i,t_i) = \mu(x_i, y_i, t_i) + \nu(x_i, y_i, t_i) $
4. Compute $\omega_i = \frac{\mu(x_i, y_i, t_i)}{\lambda(x_i,y_i,t_i)}$
5. Use $\omega_i $ to compute ML estimate of K
6. Finally, use computed K to find the background rate $ \mu(x, y, t) $ using centroid of study area as reference point, using time steps of 1 day

### Equations
$$ \lambda(x,y,t) = \mu(x,y,t) + \nu(x,y,t) $$ <br>
where $\lambda(x,y,t)$ is the total seismicity rate, $\mu(x,y,t)$ is the background seismicity rate, and $\nu(x,y,t)$ is the triggering rate <br>

The triggering rate: <br>
$$ \nu(x,y,t) = \displaystyle\sum_{i/t_i < t}^{} \frac{Ke^{\alpha m_i}}{(t+c-t_i)} \times \frac{\gamma - 1}{2\pi} \times \frac{L_i^{\gamma-1}}{\left((x-x_i)^2 + (y-y_i)^2 + L_i^2 \right )^{\frac{\gamma + 1}{2}}} $$ <br>

The background rate: <br>
$$ \mu(x,y,t) = \displaystyle\sum_{i}^{} \omega_i e^{-\sqrt{(x-x_i)^2 + (y-y_i)^2}/\ell} e^{-|t-t_i|/\tau} \times \frac{1}{2 \pi \ell^2 a_i} $$ <br>

$$ a_i = 2\tau - \tau \left( e^{-\frac{t_s - t_i}{\tau}} - e^{\frac{t_s - t_i}{\tau}} \right) $$ <br>
where $t_s, t_e$ are the start and end times of the catalog

In [ ]:
# Prepare the catalog for processing
def prep_catalog(cat_init, cat_start, cat_start): 
    ############################################
    # Prepares the catalog for further processing
    # input cat_init needs to be a pandas dataframe, preferably a geodataframe with cols being:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag, datetime
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    ###########################################
    # translate target lat, lon to radians for spherical distance calculation
    cat_init['lat_rad'] = np.radians(cat_init['lat'])
    cat_init['lon_rad'] = np.radians(cat_init['lon'])
    # Compute the time difference between event occurrence times and the start and end times of the catalog, in days
    cat_init['t_diff_e'] = (1./(24.*60.*60.))*((cat_start - cat_init['datetime']).dt.total_seconds())
    cat_init['t_diff_s'] = (1./(24.*60.*60.))*((cat_start - cat_init['datetime']).dt.total_seconds())
    return cat_init

In [ ]:
# Characteristic length/rupture radius in km
def L_i(m):
    return np.power(10, 0.5*(m-2))

# a coefficients
def a_coeff(t_diff_e,t_diff_s,tau): # tau is the temporal smoothing param; t_diff_e=t_catend-tevent; t_diff_s=t_catstart-tevent
    a = 2*tau - tau*(np.exp(-(t_diff_s/(tau))) - np.exp(-(t_diff_e/(tau)))) # times in days
    return a

# Calculate triggering rate
def nu_calc(c, alpha, gamma, K, m, time_diffs, r_sq):
    # Numerical calculations for nu
    T1 = K*np.exp(alpha*m)/(time_diffs)
    #T1 = np.exp(alpha*m)
    T2 = (gamma-1)/2*np.pi
    T3 = np.power(L_i(m), (gamma-1))
    T4 = np.power((r_sq + np.power(L_i(m),2)), (gamma+1)/2)
    return T1*T2*(T3/T4)
    
# Calculate the background rate:
def mu_calc(r_sq, t_diff, omega, tau, l, a_coeffs): # tau, l are the temporal and spatial smoothing params
    T1 = omega*np.exp(-np.sqrt(r_sq)/l)
    T2 = np.exp(-np.abs(t_diff)/tau) # times need to be in days
    T3 = 1/(2*np.pi*(l**2)*a_coeffs)
    lam = T1*T2*T3
    return lam

# Triggering rate from catalog
def nuK(catalog, c, alpha, gamma, K):
    # Calculates nu(xi,yi,ti) - triggering rate at time and place of each event in catalog
    # If K=1, then function actually returns nu(xi,yi,ti)/K
    calc_start = datetime.now() # time the function
    cat = catalog.copy(deep=True)
    # translate target lat, lon to radians for spherical distance calculation
    #cat['lat_rad'] = np.radians(cat['lat'])
    #cat['lon_rad'] = np.radians(cat['lon'])
    cat['nuK'] = 0.0
    print('Looping through catalog...')
    for triggered in cat.head(-1).itertuples():
        # get values of source event
        ttime = triggered.datetime
        #print('Triggered event time:', ttime)
        tlatrad = triggered.lat_rad
        tlonrad = triggered.lon_rad
        potential_triggers = cat.loc[cat["datetime"] < ttime]
        potential_triggers['c'] = c
        potential_triggers['t_diffs'] = (1./(24.*60.*60.))*(ttime - potential_triggers['datetime']).dt.total_seconds()
        potential_triggers['t_denom'] = potential_triggers['t_diffs'] + potential_triggers['c']
        #print(potential_triggers['t_denom'])
        #print(potential_triggers)
        # Calculate distances between triggered and potential triggers
        potential_triggers['r_squared'] = np.square(haversine(tlatrad,potential_triggers['lat_rad'],tlonrad,potential_triggers['lon_rad']))
        #print(len(potential_triggers['mag']), len(potential_triggers['r_squared']), len(potential_triggers['t_denom']))
        # Calculate triggering rate nu for each event i.e. nu(xi,yi,ti)
        nuK_array = nu_calc(c, alpha, gamma, K, potential_triggers['mag'], potential_triggers['t_denom'], potential_triggers['r_squared'])
        cat.loc[triggered.Index, 'nuK'] = nuK_array.sum()
    print('    took', (datetime.now() - calc_start), 'to compute nu(xi,yi,ti)/K \n')
    return cat

# Background rate from catalog
def mu(catalog, tau, l):
    cat = catalog.copy(deep=True)
    #cat['lat_rad'] = np.radians(cat['lat'])
    #cat['lon_rad'] = np.radians(cat['lon'])
    cat['mu_i'] = 0.0
    cat['omega_initial'] = 0.5
    for event in cat.head(-1).itertuples():
        temp_cat = cat.copy(deep=True)
        evtime = event.datetime
        print(evtime)
        #print('Triggered event time:', ttime)
        evlatrad = event.lat_rad
        evlonrad = event.lon_rad
        temp_cat['t_diffs'] = (1./(24.*60.*60.))*((evtime - temp_cat['datetime']).dt.total_seconds())
        # Calculate distances between triggered and potential triggers
        temp_cat['r_squared'] = np.square(haversine(evlatrad,temp_cat['lat_rad'],evlonrad,temp_cat['lon_rad']))
        temp_cat['a_coeffs'] = a_coeff(temp_cat['t_diff_e'], temp_cat['t_diff_s'], tau)
        temp_cat['mu_indiv'] = mu_calc(temp_cat['r_squared'],temp_cat['t_diffs'],temp_cat['omega_initial'],tau,l,temp_cat['a_coeffs'])
        print(temp_cat)
        cat.loc[event.Index, 'mu_i'] = temp_cat['mu_indiv'].sum()
    return cat

In [ ]:
#returned_cat = nuK(catalog_gdf_d_filter, c=0.001, alpha=2, gamma=2, K=1)

In [ ]:
#print(returned_cat)

In [ ]:
#start_time = datetime(1970,1,1)
#end_time = datetime(2021, 6, 21)
#cat_stats = mu(returned_cat, 100.0, 100.0)

In [ ]:
#print(cat_stats)

In [ ]:
## MLE estimate of parameter K ##

# Calculate parameter F_i for individual events
def F_i(alpha, t_diff, c, m):
    return np.exp(alpha*m)*(np.log(t_diff) - np.log(c))

# MLE estimate of the normalisation parameter K
def K_param(catalog, c, alpha): # cat needs to contain mu, lambda for each event
    calc_start = datetime.now() # time the function
    cat = catalog.copy(deep=True)
    cat['c_secs'] = c # in days
    cat['t_diffs'] = (ttime - cat['datetime']).dt.total_seconds()
    cat['t_quantity'] = cat['t_diffs'] + cat['c']
    cat['K_num'] = 1 - (cat['omega_i'])
    cat['K_denom'] = F_i(alpha, cat['t_quantity'], c, cat['mag'])
    K = cat['K_num'].sum() / cat['K_denom'].sum
    return K

In [ ]:
## Declustering function ##
def decluster(cat, cat_start, cat_end, tau, l, c, alpha, gamma):
    #################################################################################################
    # Function to decluster a catalog and provide probability of events belonging to the background
    # cat is a pandas/geopandas dataframe containing cols:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag, datetime
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    # tau, l are temporal and spatial smoothing params, to be given in days and km
    # c, alpha, gamma are Omori-Utsu Law and power spectral density constants
    ##################################################################################################
    assert t_cat_start < t_cat_end # Catalog start time must be earlier than the catalog end time
    
    # Prepare the catalog for processing by adding lat, lon in rad and adding time differences to the start and end of catalog
    cat_preprocessed = prep_catalog(cat_start, cat_end, cat_start)
    
    # Now calculate nu(xi,yi,ti)/K for all events:
    nuK_cat = nuK(cat_preprocessed, c, alpha, gamma, K=1)
    
    # Calculate background rates at time and place of each event, assuming omega is 0.5
    initial_mu_cat = mu(catalog, tau, l)
    
    # initial_mu_cat should now contain both a nu and a mu for each event
    # Now compute omega:
    initial_mu_cat['lambda_i'] = initial_mu_cat['mu_i'] + initial_mu_cat['nuK']
    initial_mu_cat['omega_i'] = initial_mu_cat['mu_i'] + initial_mu_cat['nuK']
    
    # Using the omegas, now perform a max likelihood estimate (MLE) for parameter K:
    K = K_param(initial_mu_cat, c, alpha)
    
    # Using the updated K value recalculate triggering rate
    final_cat = nuK(initial_mu_cat, c, alpha, gamma, K)
    
    # now the final catalog should contain the correct omegas, which can be used to estimate a background seismicity rate curve
    # Check this visually using a histogram
    plt.hist(final_cat["omega_i"],log=True)
    return final_cat

In [ ]:
## Final estimate of background seismicity rate ##
def mu_final(x,y,cat_start, cat_end, cat, tau, l):
    #########################################################
    # Function computes timeseries of the background seismicity rate
    # x,y refer to a spatial reference point - should be the centroid of the study area
    # Function will build an array of datetime objects with a timestep of 1 day using the cat_start, cat_end times
    # catalog should contain omega, a_coeff values for each event - the output of func decluster
    #########################################################
    assert t_cat_start < t_cat_end # Catalog start time must be earlier than the catalog end time
    # Time steps for calculating background rate
    times = np.arange(start_time, end_time, timedelta(days=1)).astype(datetime)
    mu_t_series = []
    # Compute background rate at each time step
    for t_step in times:
        temp_cat = cat.copy(deep=True)
        temp_cat['t_diffs'] = (1./(24.*60.*60.))*((t_step - temp_cat['datetime']).dt.total_seconds())
        # Calculate distances between triggered and potential triggers
        temp_cat['r_squared'] = np.square(haversine(x, temp_cat['lat_rad'], y, temp_cat['lon_rad']))
        temp_cat['mu_indiv'] = mu_calc(temp_cat['r_squared'],temp_cat['t_diffs'],temp_cat['omega_i'],tau,l,temp_cat['a_coeffs'])
        mu_t_series.append(temp_cat['mu_indiv'].sum())
    return times, mu_t_series

#### Declustering implementation

In [357]:
# ISC web search params
start_time = datetime(1970,1,1)
end_time = datetime(2021, 6, 21)
min_latitude = -25
max_latitude = -11
min_longitude = -80
max_longitude = -66
min_mag = 4.5
max_mag = None
min_depth = None
max_depth = None
print(start_time, end_time)

1970-01-01 00:00:00 2021-06-21 00:00:00


In [359]:
# Get spatial reference point
from shapely.geometry import Polygon

lat_point_list = [min_latitude, max_latitude, min_latitude]
lon_point_list = [min_longitude, max_longitude, min_longitude]

search_area = Polygon(zip(lon_point_list, lat_point_list))
crs = {'init': 'epsg:4326'}
search_area = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
#print(polygon.geometry)

x_cent = np.radians(search_area.centroid.x)
y_cent = np.radians(search_area.centroid.y)
print('Geographic ref point: ', x_cent, y_cent)

Geographic ref point:  0   -1.27409
dtype: float64 0   -0.314159
dtype: float64


C:\Users\Savvas Marcou\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if sys.path[0] == '':
C:\Users\Savvas Marcou\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  del sys.path[0]


In [ ]:
### DECLUSTERING ###
# Separate catalog into deep and shallow following Jara et al. (2017)
catalog_shallow = catalog_gdf.loc[catalog_gdf['depth_km'] < 40.0]
catalog_deep = catalog_gdf.loc[catalog_gdf['depth_km'] > 80.0]
declustered_shallow_cat = decluster(catalog_shallow, start_time, end_time, tau=100, l=100, c=0.001, alpha=2, gamma=2)
declustered_deep_cat = decluster(catalog_deep, start_time, end_time, tau=100, l=100, c=0.001, alpha=2, gamma=2)

# And calculate rates:
times, deep_rate = mu_final(x_cent,y_cent,start_time, end_time, declustered_deep_cat, tau=100, l=100)
times, shallow_rate = mu_final(x_cent,y_cent,start_time, end_time, declustered_shallow_cat, tau=100, l=100)

In [ ]:
#### PLOTTING ####